# IR Lab SoSe 2024: Baseline Retrieval System

This jupyter notebook serves as baseline retrieval system that you can try to improve upon.
We will use the a corpus of scientific papers (title + abstracts) from the fields of information retrieval and natural language processing (the [IR Anthology](https://ir.webis.de/anthology/) and the [ACL Anthology](https://aclanthology.org/)). This serves Jupyter notebook only serves as retrieval system, i.e., it gets a set of information needs (topics) and a corpus as input and produces a run file as output. Please do evaluations in a new dedicated notebook.

### Step 1: Import Libraries

We will use [tira](https://www.tira.io/), an information retrieval shared task platform, for loading the (pre-built) retrieval index and [ir_dataset](https://ir-datasets.com/) to subsequently build a retrieval system with [PyTerrier](https://github.com/terrier-org/pyterrier), an open-source search engine.

Building your own index can be already one way that you can try to improve upon this baseline (if you want to focus on creating good document representations). Other ways could include reformulating queries or tuning parameters or building better retrieval pipelines.

In [1]:
import pyterrier as pt


# Initialize PyTerrier with the custom JAR file
pt.init(
    version='5.9',  # specify the version corresponding to your custom JAR
    home_dir=None,  # set to None if JAR is in ~/.pyterrier
    packages=['org.terrier:terrier-project:5.9'],
    no_download=True
)

from pyterrier.index import IterDictIndexer
# Your PyTerrier code goes here
print("PyTerrier initialized with custom Terrier JAR!")


#index = tira.pt.index('ir-lab-sose-2024-needthegrade/baseline-retrieval-system/index_ngram2', test_dataset)



# Initialize PyTerrier


# Create an instance of the custom indexer
index_path = "./my_custom_index"
custom_indexer = IterDictIndexer(index_path,stemmer=None, stopwords=None, tokeniser='UTF2Tokeniser') 
# hier kann man den actual java tokeniser auswählen. sieht man an wörter mit umlaute, die bei utftokeniser genommen werden
# aber nicht bei engliushtoeniser. jetzt nur noch schauen wie wir auf den neuen UTFNGramTokeniser zugreifen können
#vlt mit pretokenised machen dass dann gar kein java tokeniser genommen wird und man es in python machen vorher kann

# Example documents to index
docs = [
    {"docno": "0", "text": "information retrieval systems for about a million times i try to do the system for the retrieval pipeline, why doesnt it work"},
    {"docno": "1", "text": "house cat ähnlich system"},
    {"docno" : "2", "text" : "systems for implementing large scale retrieval systems"}
]

# Index the documents
#custom_indexer.index.setproperties("")
index_ref = custom_indexer.index(docs)

# Now you can use the index_ref as usual
index = pt.IndexFactory.of(index_ref)

print(index.getCollectionStatistics())
for term, le in index.getLexicon():
    print(term) 
    print(le.getFrequency())

# Access the MetaIndex and Lexicon
meta = index.getMetaIndex()
lexicon = index.getLexicon()



PyTerrier 0.10.0 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


09:42:23.543 [main] DEBUG org.eclipse.aether.internal.impl.DefaultLocalRepositoryProvider - Using manager EnhancedLocalRepositoryManager with priority 10.0 for /root/.m2/repository
09:42:23.795 [main] DEBUG org.eclipse.aether.internal.impl.DefaultTransporterProvider - Using transporter HttpTransporter with priority 5.0 for https://repo.maven.apache.org/maven2/
09:42:23.795 [main] DEBUG org.eclipse.aether.internal.impl.DefaultRepositoryConnectorProvider - Using connector BasicRepositoryConnector with priority 0.0 for https://repo.maven.apache.org/maven2/
09:42:23.845 [main] DEBUG org.apache.http.impl.conn.PoolingClientConnectionManager - Connection request: [route: {s}->https://repo.maven.apache.org:443][total kept alive: 0; route allocated: 0 of 50; total allocated: 0 of 100]
09:42:23.850 [main] DEBUG org.apache.http.impl.conn.PoolingClientConnectionManager - Connection leased: [id: 0][route: {s}->https://repo.maven.apache.org:443][total kept alive: 0; route allocated: 1 of 50; total a

JavaException: JVM exception occurred: java.util.concurrent.ExecutionException: java.lang.RuntimeException: java.lang.RuntimeException: Problem with JSON Document java.lang.RuntimeException

In [2]:
# Imports
#from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
#from tira.rest_api_client import Client
#import pyterrier as pt
#from pyterrier.index import IterDictIndexer
#import pandas as pd #extra importfrom sklearn.feature_extraction.text import CountVectorizer #extra import

### Step 2: Load the Dataset and the Index

The type of the index object that we load is `<class 'jnius.reflect.org.terrier.structures.Index'>`, in fact a [Java class](http://terrier.org/docs/v3.6/javadoc/org/terrier/structures/Index.html) wrapped into Python. However, you do not need to worry about this: at this point, we will simply use the provided Index object to run procedures defined in Python.

Create a custom tokenizer method.
This method takes a String as an argument and returns a List of all tokens, including ngrams between 1 and 3 words.

Create the Iterator for our dataset. This is an Iterator where every element is a dict which contains a text and a docno.
See the example print below.

### Step 3: Define the Retrieval Pipeline

We will define a BM25 retrieval pipeline as baseline. For details, see:

- [https://pyterrier.readthedocs.io](https://pyterrier.readthedocs.io)
- [https://github.com/terrier-org/ecir2021tutorial](https://github.com/terrier-org/ecir2021tutorial)

In [2]:
bm25 = pt.BatchRetrieve(index_ref, wmodel="BM25")

### Step 4: Create the Run


In [3]:
print('Now we do the retrieval...')
#run = bm25(test_dataset.get_topics('text'))
run = bm25([{"qid": "0", "query" : "retrieval systems"}])
print('Done. Here are the first 10 entries of the run')
run.head(10)

Now we do the retrieval...
Done. Here are the first 10 entries of the run


,qid,docid,docno,rank,score,query
0,0,0,0,0,-1.313898,retrieval systems
1,0,2,2,1,-1.994526,retrieval systems


In [9]:
#index = pt.IndexFactory.of(index_ref)
meta = index.getMetaIndex()

# List of metadata keys
meta_keys = ['docno', 'text']  # Adjust this list based on your meta fields

# Function to print document attributes
def print_doc_attributes(docid):
    attributes = {key: meta.getItem(key, docid) for key in meta_keys}
    print(f"Attributes for docid {docid}: {attributes}")

# Example: Print attributes for the first 5 documents
for docid in range(3):
    print_doc_attributes(docid)

# Function to print document attributes by docno
def print_doc_attributes_by_docno(docno):
    try:
        docid = meta.getDocument("docno", docno)
        print_doc_attributes(docid)
    except KeyError:
        print(f"Document with docno {docno} not found.")

# List of specific docnos to retrieve
#docnos = ['W05-0704']

# Retrieve and print attributes for each specified docno
#for docno in docnos:
 #   print_doc_attributes_by_docno(docno)

Attributes for docid 0: {'docno': '0', 'text': ''}
Attributes for docid 1: {'docno': '1', 'text': ''}
Attributes for docid 2: {'docno': '2', 'text': ''}


### Step 5: Persist the run file for subsequent evaluations

The output of a prototypical retrieval system is a run file. This run file can later (optimally in a different notebook) be statistically evaluated.

In [ ]:
persist_and_normalize_run(run, system_name='bm25-baseline', default_output='../runs')

The run file is normalized outside the TIRA sandbox, I will store it at "../runs".
Done. run file is stored under "../runs/run.txt".
